In [1]:
# import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
# import our first source file

# source file 1, the whole neighborhood file
Beverly_Grove_Total_df = pd.read_csv('Neighborhood_Files/Beverly_Grove_Finished.csv', index_col='PARCEL')
Beverly_Grove_Total_df.head()

,Composite Name,Owned by Trust?,Owned by Business?,SITEADDRESS,SITECITY,SITEZIP,MAILADDRESS,MAIL DIFFERENT FROM SITE?,MAILCITY,MAILSTATE,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,WALLHGT,VIEW
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,Mark K Nishida Sherman Dale S,No,No,1030 S Dunsmuir Ave,Los Angeles,90019,1030 S Dunsmuir Ave,No,Los Angeles,CA,...,2.0,0,1,0,0,YES,CENTRAL,CENTRAL,0,0
5085033008,B Erie & Fay Marshall,No,No,1364 S Ridgeley Dr,Los Angeles,90019,1942 E Woodgate Dr,Yes,West Covina,CA,...,1.0,0,0,0,0,0,0,YES,0,0
5085032022,Neal S Co Scheindlin Scheindlin & Weisber,No,No,1322 Hauser Blvd,Los Angeles,90019,1322 Hauser Blvd,No,Los Angeles,CA,...,1.0,0,0,0,0,0,0,YES,0,0
5510005006,Simi Dabah The Realty Trust,Yes,No,6727 Drexel Ave,Los Angeles,90048,2068 Orange Tree Ln #100,Yes,Redlands,CA,...,5.0,0,1,0,0,YES,0,CENTRAL,0,0
5089024002,Frances Travassaros Veracka Virginia,No,No,942 S Cochran Ave,Los Angeles,90036,942 S Cochran Ave,No,Los Angeles,CA,...,1.0,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0


In [3]:
# import our second source file

# source file 2, the sales for that neighborhood
Beverly_Grove_Sales_df = pd.read_csv('Sales_Files/Beverly_Grove_Sales.csv', index_col='PARCEL')
Beverly_Grove_Sales_df.head()

,Owner Name2,Owner1 First Name,Owner1 Last Name,Site Address,Site Address House Number,Site Address Unit Number,Site Address Street Name,Site Address City,Site Address State,Site Address Zip,...,Building Area,Sales Price,Tax Delinquent,Tax Exemption Code,Tract,Use Code,Use Code Description,Census Tract,Year Built,Zoning
PARCEL,,,,,,,,,,,,,,,,,,,,,
5089002004,Astrid Meghrigian,Astrid,Meghrigian,NaN,NaN,NaN,NaN,Los Angeles,CA,90036,...,6100.0,NaN,N,None,5691.0,2036,Parking Lot,2151.021000,NaN,Lac2
5086013015,6000 Sv Holdings Llc,NaN,6000 Sv Holdings Llc,NaN,NaN,NaN,NaN,Los Angeles,CA,90036,...,5000.0,20000000.0,N,None,9721.0,2036,Parking Lot,2169.003001,1970.0,Lac2
5512016010,Scott Strumwasser & Mahtash Rahbar,Scott,Strumwasser,5971 W 3Rd St,5971.0,NaN,3Rd St,Los Angeles,CA,90036,...,9487.0,NaN,N,None,7372.0,2001,"Retail Stores ( Personal Services, Photography...",2140.002009,1929.0,Lac1
5511028030,Hart Third Street Llc,NaN,Hart Third Street Llc,8038 W 3Rd St,8038.0,NaN,3Rd St,Los Angeles,CA,90048,...,2205.0,NaN,N,None,7555.0,2001,"Retail Stores ( Personal Services, Photography...",2146.002001,1946.0,Lac2
5511029032,Triple Heights Llc,NaN,Triple Heights Llc,8124 W 3Rd St,8124.0,NaN,3Rd St,Los Angeles,CA,90048,...,6400.0,8700000.0,N,None,7555.0,3003,Office Bldg (General),2146.002002,1967.0,Lac2


In [4]:
# create a new column which will later be transformed into our output column after we've merged the two df's

Beverly_Grove_Sales_df['Did it sell?'] = 'yes'

# check the new column
Beverly_Grove_Sales_df.head()

,Owner Name2,Owner1 First Name,Owner1 Last Name,Site Address,Site Address House Number,Site Address Unit Number,Site Address Street Name,Site Address City,Site Address State,Site Address Zip,...,Sales Price,Tax Delinquent,Tax Exemption Code,Tract,Use Code,Use Code Description,Census Tract,Year Built,Zoning,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5089002004,Astrid Meghrigian,Astrid,Meghrigian,NaN,NaN,NaN,NaN,Los Angeles,CA,90036,...,NaN,N,None,5691.0,2036,Parking Lot,2151.021000,NaN,Lac2,yes
5086013015,6000 Sv Holdings Llc,NaN,6000 Sv Holdings Llc,NaN,NaN,NaN,NaN,Los Angeles,CA,90036,...,20000000.0,N,None,9721.0,2036,Parking Lot,2169.003001,1970.0,Lac2,yes
5512016010,Scott Strumwasser & Mahtash Rahbar,Scott,Strumwasser,5971 W 3Rd St,5971.0,NaN,3Rd St,Los Angeles,CA,90036,...,NaN,N,None,7372.0,2001,"Retail Stores ( Personal Services, Photography...",2140.002009,1929.0,Lac1,yes
5511028030,Hart Third Street Llc,NaN,Hart Third Street Llc,8038 W 3Rd St,8038.0,NaN,3Rd St,Los Angeles,CA,90048,...,NaN,N,None,7555.0,2001,"Retail Stores ( Personal Services, Photography...",2146.002001,1946.0,Lac2,yes
5511029032,Triple Heights Llc,NaN,Triple Heights Llc,8124 W 3Rd St,8124.0,NaN,3Rd St,Los Angeles,CA,90048,...,8700000.0,N,None,7555.0,3003,Office Bldg (General),2146.002002,1967.0,Lac2,yes


In [5]:
# drop the useless columns, i.e only keep 'PARCEL' and 'Did it sell?'

Beverly_Grove_Sales_df = Beverly_Grove_Sales_df[['Did it sell?']]

In [6]:
# check to see if it worked.

Beverly_Grove_Sales_df.head()

,Did it sell?
PARCEL,
5089002004,yes
5086013015,yes
5512016010,yes
5511028030,yes
5511029032,yes


# Merge our dataframes

In [7]:
Beverly_Grove_Total_df = Beverly_Grove_Total_df.merge(Beverly_Grove_Sales_df, how='left', left_index=True, right_index=True)
Beverly_Grove_Total_df.head()

,Composite Name,Owned by Trust?,Owned by Business?,SITEADDRESS,SITECITY,SITEZIP,MAILADDRESS,MAIL DIFFERENT FROM SITE?,MAILCITY,MAILSTATE,...,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,WALLHGT,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,Mark K Nishida Sherman Dale S,No,No,1030 S Dunsmuir Ave,Los Angeles,90019,1030 S Dunsmuir Ave,No,Los Angeles,CA,...,0,1,0,0,YES,CENTRAL,CENTRAL,0,0,NaN
5085033008,B Erie & Fay Marshall,No,No,1364 S Ridgeley Dr,Los Angeles,90019,1942 E Woodgate Dr,Yes,West Covina,CA,...,0,0,0,0,0,0,YES,0,0,NaN
5085032022,Neal S Co Scheindlin Scheindlin & Weisber,No,No,1322 Hauser Blvd,Los Angeles,90019,1322 Hauser Blvd,No,Los Angeles,CA,...,0,0,0,0,0,0,YES,0,0,NaN
5510005006,Simi Dabah The Realty Trust,Yes,No,6727 Drexel Ave,Los Angeles,90048,2068 Orange Tree Ln #100,Yes,Redlands,CA,...,0,1,0,0,YES,0,CENTRAL,0,0,NaN
5089024002,Frances Travassaros Veracka Virginia,No,No,942 S Cochran Ave,Los Angeles,90036,942 S Cochran Ave,No,Los Angeles,CA,...,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0,NaN


In [8]:
# Fill in the NaN's in the 'Did it sell' column
Beverly_Grove_Total_df['Did it sell?'] = Beverly_Grove_Total_df['Did it sell?'].fillna('no')

In [9]:
# check the value counts
Beverly_Grove_Total_df['Did it sell?'].value_counts()

no     6228
yes     502
Name: Did it sell?, dtype: int64

# Drop the columns we don't need

In [10]:
# check columns
Beverly_Grove_Total_df.columns

Index(['Composite Name', 'Owned by Trust?', 'Owned by Business?',
       'SITEADDRESS', 'SITECITY', 'SITEZIP', 'MAILADDRESS',
       'MAIL DIFFERENT FROM SITE?', 'MAILCITY', 'MAILSTATE',
       'MAIL OUTSIDE CA?', 'TITLECO1', 'ASSDTOTAL', 'ASSDLAND', 'ASSDSTCT',
       'ASSDOTHR', 'EXEMPTCD', 'EXMPTAMT', 'PCNTIMPD', 'TAXAMT',
       'DOCDATE_YEAR', 'DEEDTYPE', 'MULTIPARCEL', 'PRICE', 'LENDER1',
       'LOANAMOUT1', 'LOANTYPE1', 'YEARBLT', 'EFFYRBLT', 'LOTSQFT', 'TOTALSF',
       'GRGTYPE', 'BLDGMAT', 'TOTUNITS', 'QUALCLAS', 'BLDGCOND', 'NOSTORY',
       'ROOFMAT', 'FOUNDATN', 'BEDROOMS', 'BATHROOMS', 'FAMILYRM', 'DININGRM',
       'POOL', 'PATIO', 'FIREPLCE', 'AIRMTHOD', 'HEATMTHD', 'WALLHGT', 'VIEW',
       'Did it sell?'],
      dtype='object')

In [11]:
# drop 'Composite Name', 'SITEADDRESS', 'SITECITY', 'SITEZIP', 'MAILADDRESS','WALLHGT'
Beverly_Grove_Total_df.drop(['Composite Name', 'SITEADDRESS', 'SITECITY', 'SITEZIP',
                             'MAILADDRESS', 'WALLHGT', 'MAILCITY', 'MAILSTATE'],
                              axis=1, inplace=True)

Beverly_Grove_Total_df.head()

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,No,No,No,No,FIRST AMERICAN TITLE,1157385,925908,231477,0,0,...,2.0,0,1,0,0,YES,CENTRAL,CENTRAL,0,no
5085033008,No,No,Yes,No,0,57047,40994,16053,0,0,...,1.0,0,0,0,0,0,0,YES,0,no
5085032022,No,No,No,No,ATTORNEY ONLY,651485,521193,130292,0,HOMEOWNERS,...,1.0,0,0,0,0,0,0,YES,0,no
5510005006,Yes,No,Yes,No,0,85697,74539,11158,0,0,...,5.0,0,1,0,0,YES,0,CENTRAL,0,no
5089024002,No,No,No,No,0,62487,42013,20474,0,HOMEOWNERS,...,1.0,1,0,0,YES,YES,CENTRAL,CENTRAL,0,no


# Encode Column Values

In [12]:
# Determine the number of unique values in each column.

Beverly_Grove_Total_df.apply(lambda x: len(x.unique()))

Owned by Trust?                 2
Owned by Business?              2
MAIL DIFFERENT FROM SITE?       2
MAIL OUTSIDE CA?                2
TITLECO1                      119
ASSDTOTAL                    6137
ASSDLAND                     5124
ASSDSTCT                     5603
ASSDOTHR                        5
EXEMPTCD                        2
EXMPTAMT                       14
PCNTIMPD                       94
TAXAMT                       6671
DOCDATE_YEAR                   58
DEEDTYPE                       23
MULTIPARCEL                     3
PRICE                        1172
LENDER1                       504
LOANAMOUT1                   1394
LOANTYPE1                      11
YEARBLT                        98
EFFYRBLT                       99
LOTSQFT                      1757
TOTALSF                      2186
GRGTYPE                         7
BLDGMAT                         9
TOTUNITS                       34
QUALCLAS                        6
BLDGCOND                        6
NOSTORY       

In [13]:
# Convert binary value counts to 0's and 1's, since most of them are 'Yes' or 'No' right now.
# Step 1: 'Owned by Trust?'

Beverly_Grove_Total_df['Owned by Trust?'].replace({"Yes": 1, "No": 0}, inplace=True)

In [14]:
# Step 2: 'Owned by Business?'
Beverly_Grove_Total_df['Owned by Business?'].replace({"Yes": 1, "No": 0}, inplace=True)

# Step 3: 'MAIL DIFFERENT FROM SITE?'
Beverly_Grove_Total_df['MAIL DIFFERENT FROM SITE?'].replace({"Yes": 1, "No": 0}, inplace=True)

# Step 4: 'MAIL OUTSIDE CA'
Beverly_Grove_Total_df['MAIL OUTSIDE CA?'].replace({"Yes": 1, "No": 0}, inplace=True)

# Step 5: 'EXEMPTCD'
Beverly_Grove_Total_df['EXEMPTCD'].replace({"HOMEOWNERS": 1}, inplace=True)

# Step 6: 'Did it sell?'
Beverly_Grove_Total_df['Did it sell?'].replace({"yes": 1, "no": 0}, inplace=True)

In [15]:
Beverly_Grove_Total_df.head()

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,FIRST AMERICAN TITLE,1157385,925908,231477,0,0,...,2.0,0,1,0,0,YES,CENTRAL,CENTRAL,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,1.0,0,0,0,0,0,0,YES,0,0
5085032022,0,0,0,0,ATTORNEY ONLY,651485,521193,130292,0,1,...,1.0,0,0,0,0,0,0,YES,0,0
5510005006,1,0,1,0,0,85697,74539,11158,0,0,...,5.0,0,1,0,0,YES,0,CENTRAL,0,0
5089024002,0,0,0,0,0,62487,42013,20474,0,1,...,1.0,1,0,0,YES,YES,CENTRAL,CENTRAL,0,0


In [16]:
Beverly_Grove_Total_df['Owned by Trust?'].value_counts()

0    5033
1    1697
Name: Owned by Trust?, dtype: int64

In [17]:
Beverly_Grove_Total_df['Owned by Trust?'].dtypes

dtype('int64')

In [18]:
Beverly_Grove_Total_df['FAMILYRM'].value_counts()

0    6443
1     284
2       3
Name: FAMILYRM, dtype: int64

# Now encode the categorical data

In [19]:
Beverly_Grove_Total_df.columns

Index(['Owned by Trust?', 'Owned by Business?', 'MAIL DIFFERENT FROM SITE?',
       'MAIL OUTSIDE CA?', 'TITLECO1', 'ASSDTOTAL', 'ASSDLAND', 'ASSDSTCT',
       'ASSDOTHR', 'EXEMPTCD', 'EXMPTAMT', 'PCNTIMPD', 'TAXAMT',
       'DOCDATE_YEAR', 'DEEDTYPE', 'MULTIPARCEL', 'PRICE', 'LENDER1',
       'LOANAMOUT1', 'LOANTYPE1', 'YEARBLT', 'EFFYRBLT', 'LOTSQFT', 'TOTALSF',
       'GRGTYPE', 'BLDGMAT', 'TOTUNITS', 'QUALCLAS', 'BLDGCOND', 'NOSTORY',
       'ROOFMAT', 'FOUNDATN', 'BEDROOMS', 'BATHROOMS', 'FAMILYRM', 'DININGRM',
       'POOL', 'PATIO', 'FIREPLCE', 'AIRMTHOD', 'HEATMTHD', 'VIEW',
       'Did it sell?'],
      dtype='object')

In [20]:
# Those columns will be the following: 
# TITLECO1
# DEEDTYPE
# MULTIPARCEL
# LENDER1
# LOANTYPE1
# GRGTYPE
# BLDGMAT
# QUALCLAS
# BLDGCOND
# POOL
# PATIO
# FIREPLCE
# AIRMTHOD
# HEATMTHD
# VIEW

In [22]:
Encoded_List = [
    'TITLECO1',
    'MULTIPARCEL',
    'LENDER1',
    'LOANTYPE1',
    'GRGTYPE',
    'BLDGMAT',
    'QUALCLAS',
    'BLDGCOND',
    'POOL',
    'PATIO',
    'FIREPLCE',
    'AIRMTHOD',
    'HEATMTHD',
    'VIEW',
]

In [23]:
for col in Beverly_Grove_Total_df:
    if col in Encoded_List:
        Beverly_Grove_Total_df[col] = Beverly_Grove_Total_df[col].transform(lambda x: x != '0')
        Beverly_Grove_Total_df[col] = Beverly_Grove_Total_df[col].astype(int)


In [28]:
Beverly_Grove_Total_df.head()

,Owned by Trust?,Owned by Business?,MAIL DIFFERENT FROM SITE?,MAIL OUTSIDE CA?,TITLECO1,ASSDTOTAL,ASSDLAND,ASSDSTCT,ASSDOTHR,EXEMPTCD,...,BATHROOMS,FAMILYRM,DININGRM,POOL,PATIO,FIREPLCE,AIRMTHOD,HEATMTHD,VIEW,Did it sell?
PARCEL,,,,,,,,,,,,,,,,,,,,,
5085010022,0,0,0,0,1,1157385,925908,231477,0,0,...,2.0,0,1,0,0,1,1,1,0,0
5085033008,0,0,1,0,0,57047,40994,16053,0,0,...,1.0,0,0,0,0,0,0,1,0,0
5085032022,0,0,0,0,1,651485,521193,130292,0,1,...,1.0,0,0,0,0,0,0,1,0,0
5510005006,1,0,1,0,0,85697,74539,11158,0,0,...,5.0,0,1,0,0,1,0,1,0,0
5089024002,0,0,0,0,0,62487,42013,20474,0,1,...,1.0,1,0,0,1,1,1,1,0,0


## Now encode 'Deedtype' but to into individual values for 9 largest, plus 'Other' and '0' values. 
